In [1]:
import pandas as pd
import sys

sys.path.append("/var/DSTeam/locnt2/personal/quotes-tagging/")
from collections import Counter
import torch
from torch import nn
import numpy as np

/var/DSTeam/DScommon/anaconda3/envs/py38DL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("/var/DSTeam/locnt2/personal/quotes-tagging/data/quotes_clean.csv")

In [3]:
df.tags = df.tags.str.lower()
df.tags = df.tags.str.split(",")

In [4]:
tags = df.tags.explode().tolist()

In [5]:
df.quote.iloc[4]

'Darkness cannot drive out darkness: only light can do that. Hate cannot drive out hate: only love can do that.'

In [6]:
keys = list(Counter(tags).keys())
vals = list(Counter(tags).values())

In [96]:
df_tags = (
    pd.DataFrame({"tag": keys, "num_quote": vals})
    .dropna(subset=["tag"])
    .sort_values("num_quote", ascending=False)
)

In [8]:
df_tags.sort_values("num_quote", ascending=False).iloc[:20].num_quote.sum()

239737

In [9]:
from transformers import BertTokenizer, BertModel

In [10]:
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [12]:
df.dropna(subset=["quote"]).loc[
    lambda x: x.quote.str.contains("self-respect, and")
].iloc[0].tags

['courage', ' discipline', ' self-control', ' self-respect']

In [13]:
tokenizer("self-respect")

{'input_ids': [101, 2969, 1011, 4847, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [14]:
string = "attributed-no-source"

In [15]:
with torch.no_grad():
    out = model(**tokenizer(string, return_tensors="pt"))

In [16]:
def text_to_vector(text):
    with torch.no_grad():
        out = model(**tokenizer(text, return_tensors="pt"))
    return out.pooler_output.numpy().flatten()

In [101]:
all_tags = df_tags.tag.tolist()

In [102]:
device = torch.device("cuda:0")
model = model.to(device)

In [103]:
vectors_tag = []

In [104]:
for x in np.array_split(all_tags, 20):
    tokenzed_tags = tokenizer(x.tolist(), return_tensors="pt", padding=True)
    tokenzed_tags = tokenzed_tags.to(device)
    with torch.no_grad():
        out = model(**tokenzed_tags)
        vectors_tag.append(out.pooler_output.cpu().numpy())
    torch.cuda.empty_cache()

In [105]:
vectors_tag_vstack = np.vstack(vectors_tag)

In [106]:
df_tags["vector_tag"] = vectors_tag_vstack.tolist()

In [28]:
from sklearn.manifold import TSNE

In [107]:
tsne = TSNE(n_components=3, random_state=42)

In [108]:
X_tsne = tsne.fit_transform(vectors_tag_vstack[:10000])

In [68]:
import matplotlib.pyplot as plt

# import plotly.express as px

In [84]:
df_tags.iloc[0].tag

'attributed-no-source'

In [111]:
df_tags[:20]

,tag,num_quote,vector_tag
3,love,33650,"[-0.864437997341156, -0.25736525654792786, -0...."
2,life,31425,"[-0.903038501739502, -0.37457799911499023, -0...."
11,inspirational,23420,"[-0.7722470760345459, -0.1867963820695877, 0.6..."
31,philosophy,14106,"[-0.9659687876701355, -0.5935049057006836, -0...."
6,truth,11424,"[-0.7429459095001221, -0.11659476906061172, 0...."
175,god,10825,"[-0.8422870635986328, -0.1923561841249466, 0.3..."
72,wisdom,10481,"[-0.7098093628883362, -0.13614177703857422, 0...."
168,humor,9488,"[-0.8484600782394409, -0.3035257160663605, 0.0..."
26,life,9084,"[-0.903038501739502, -0.37457799911499023, -0...."
2477,quotes,9054,"[-0.7491070032119751, -0.18356673419475555, 0...."


In [117]:
df_tags.loc[lambda x: x.tag.str.contains("love")]

,tag,num_quote,vector_tag
3,love,33650,"[-0.864437997341156, -0.25736525654792786, -0...."
25,love,8763,"[-0.864437997341156, -0.25736525654792786, -0...."
1209,love-quotes,3883,"[-0.8061947226524353, -0.30500948429107666, -0..."
1022,self-love,1893,"[-0.8109254837036133, -0.27084362506866455, -0..."
670,lovers,1339,"[-0.8154675364494324, -0.2330590784549713, 0.3..."
...,...,...,...
89710,love-based-psychology,1,"[-0.9145378470420837, -0.5261462330818176, -0...."
90118,inspirational-love-story,1,"[-0.8933933973312378, -0.450512170791626, -0.9..."
90111,spiritual-mother-love,1,"[-0.9051916599273682, -0.3471352458000183, -0...."
90177,wisdom-love,1,"[-0.8978511691093445, -0.34850549697875977, -0..."


In [118]:
from sklearn.cluster import KMeans, DBSCAN

In [127]:
kmean = KMeans(n_clusters=30)
kmean.fit(vectors_tag_vstack[:10000])

/var/DSTeam/DScommon/anaconda3/envs/py38DL/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



KMeans(n_clusters=30)

In [128]:
kmean.labels_

array([19, 19,  0, ..., 29, 21,  2], dtype=int32)

In [171]:
list(filter(lambda x: " patien" in x, all_tags))

[' patience',
 ' patient',
 ' patience-and-timing',
 ' patience-johnson',
 ' patience-quotes',
 ' patience-and-love',
 ' patients',
 ' patient-in-life',
 ' patient ',
 ' patience-moves-mountains',
 ' patience-brings-strength',
 ' patient-quotes',
 ' patient-problems',
 ' patience-and-criticism',
 ' patience ',
 ' patient-zero',
 ' patient-people',
 ' patienit',
 ' patience-sparrow',
 ' patiently ',
 ' patient-protection',
 ' patient-love',
 ' patient-care',
 ' patience-thankfulness-power',
 ' patience-kindness',
 ' patient-endurance',
 ' patients-get-better',
 'and most importantly: our ability to listen with patience and understanding is the measure of our love',
 ' patience-righteousness',
 ' patien']

In [1]:
categories = {
    "hard times": [
        "heartbroken",
        "breakup",
        "overcoming fear",
        "be strong",
        "overthinking",
        "death",
        "haters",
        "toxic relationships",
        "depression",
        "loneliness",
        "dealing with change",
        "unrequited love",
        "uncertainty",
        "sadness",
        "missing someone",
        "dealing with frustation",
    ],
    "personal growth": [
        "letting go",
        "self-respect",
        "self-esteem",
        "be yourself",
        "ego",
        "self-developement",
        "start change",
        "be strong",
        "positive thinking",
        "happiness",
        "growth",
        "self-love",
        "new beginnings",
        "love yourself",
        "gratitude",
        "moving on",
    ],
    "work & productivity": [
        "focus",
        "graduation",
        "college",
        "study",
        "success",
        "passion",
        "money",
        "business",
        "leadership",
        "hustling",
        "work",
        "self-discipline",
        "routine",
        "entrepreneurs",
        "productivity",
        "perseverance",
    ],
    "inspiration": [
        "african-american",
        "proverbs",
        "funny",
        "future",
        "beauty",
        "passion",
        "feeling sassy",
        "women's empowerment",
        "shore quotes",
        "enjoy the moment",
        "sayings",
        "encouraging words",
        "feeling blessed",
        "art",
    ],
    "spiritual & philosophy": [
        "taoism",
        "buddhism",
        "zen",
        "devotions",
        "life",
        "karma",
        "mindfulness",
        "mantras",
        "wisdom",
        "stoicism",
        "god",
        "feeling blessed",
        "faith",
        "hope",
        "bible verses",
        "philosophy",
    ],
}

In [2]:
import json

In [3]:
with open(
    "/var/DSTeam/locnt2/personal/quotes-tagging/data/category.json", "w"
) as outfile:
    json.dump(categories, outfile)